In [2]:
# export
from deeppavlov import configs,build_model,train_model
import json
from os import path
import pandas as pd
import numpy as np

10:11:25 INFO:test: hola
10:11:25 ERROR:test: adios


In [ ]:
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logging.info("test: hola")
logging.error('test: adios')

In [3]:
!pip install deeppavlov > /dev/null
!python -m deeppavlov install squad > /dev/null
!python -m deeppavlov install squad_bert > /dev/null
!python -m deeppavlov install fasttext_avg_autofaq > /dev/null
!python -m deeppavlov install fasttext_tfidf_autofaq > /dev/null
!python -m deeppavlov install tfidf_autofaq > /dev/null
!python -m deeppavlov install tfidf_logreg_autofaq > /dev/null
!python -m deeppavlov install tfidf_logreg_en_faq > /dev/null

2020-06-09 22:11:35.793 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad' as '/opt/conda/lib/python3.7/site-packages/deeppavlov/configs/squad/squad.json'
2020-06-09 22:11:39.555 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/opt/conda/lib/python3.7/site-packages/deeppavlov/configs/squad/squad_bert.json'
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-n6xueuk8
  Running command git checkout -b feat/multi_gpu --track origin/feat/multi_gpu
  Switched to a new branch 'feat/multi_gpu'
  Branch 'feat/multi_gpu' set up to track remote branch 'feat/multi_gpu' from 'origin'.
2020-06-09 22:11:48.138 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'fasttext_avg_autofaq' as '/opt/conda/lib/python3.7/site-packages/deeppavlov/configs/faq/fasttext_avg_autofaq.json'
2020-06-09 22:11:51.207 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'fasttext_t

# Dialog System
> Question Answering Automatic System

In [4]:
#export
def replacement_f(model_config,**args):
    for k,v in args.items():
        if isinstance(v,dict):
            replacement_f(model_config[k],**v)
        else:
            model_config[k] = v
            
def gen_faq_config_file(**args):

    #set FAQ data in config file
    model_config = json.load(open(configs.faq.tfidf_logreg_en_faq))
    if 'data_url' in model_config['dataset_reader']:
        del model_config['dataset_reader']['data_url']
    
    replacement_f(model_config,**args)

    json.dump(model_config, open(configs.faq.tfidf_logreg_en_faq, 'w'))

In [5]:
#get notebook directory
current_dir = !pwd

In [6]:
faq_data_file =  path.join(current_dir[0], 'faq_data.csv')
gen_faq_config_file(
    dataset_reader={'data_path':faq_data_file}
)

In [7]:
# export
def load_qa_models():
    qa_models = {
        'squad':
            {
                'rnet': build_model(configs.squad.squad, download=True),
                'bert': build_model(configs.squad.squad_bert, download=True)
            },
        'faq':
            {
                'tfidf':
                    train_model(configs.faq.tfidf_logreg_en_faq, download=True)
            }
    }
    return qa_models

In [8]:
qa_models = load_qa_models()

10:12:01 DEBUG:Starting new HTTP connection (1): files.deeppavlov.ai:80
10:12:01 DEBUG:http://files.deeppavlov.ai:80 "GET /embeddings/wiki-news-300d-1M.vec.md5 HTTP/1.1" 200 56
2020-06-09 22:12:19.500 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M.vec download because of matching hashes
10:12:19 INFO:Skipped http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M.vec download because of matching hashes
10:12:19 DEBUG:Starting new HTTP connection (1): files.deeppavlov.ai:80
10:12:19 DEBUG:http://files.deeppavlov.ai:80 "GET /embeddings/wiki-news-300d-1M-char.vec.md5 HTTP/1.1" 200 61
2020-06-09 22:12:20.53 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M-char.vec download because of matching hashes
10:12:20 INFO:Skipped http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M-char.vec download because of matching hashes
10:12:20 DEBUG:Starting new HT

In [32]:
#export
def select_squad_responses(
    contexts, squad_model, question, num_returned_values=1
):
    responses = contexts.context.apply(
        lambda context: squad_model([context], [question])
    ).values
    return [
        r[0][0] for r in sorted(responses, key=lambda x: -1 * x[2][0])
        [:num_returned_values]
    ]


def select_faq_responses(faq_model, question):
    return faq_model([question])[0]


def format_responses(question, responses):
    formatted_response = f'{question}:\n\n'
    for k, res in enumerate(responses):
        formatted_response += f'{k}: {res}\n'
    return formatted_response


def get_responses(
    contexts, question, qa_models, num_returned_values_per_squad_model=1
):
    responses = []
    for squad_model in qa_models['squad'].values():
        responses.extend(
            select_squad_responses(
                contexts,
                squad_model,
                question,
                num_returned_values=num_returned_values_per_squad_model
            )
        )
    for faq_model in qa_models['faq'].values():
        responses.extend(select_faq_responses(faq_model, question))
    return format_responses(question, set([r for r in responses if r.strip()]))

In [36]:
#tests
example_contexts = pd.DataFrame(
    {
        'context':
            [
                'Intekglobal has its headquarters located in TJ',
                'In Intekglobal we care about you',
                'Intekglobal is in the north of mexico'
            ]
    },
)

assert select_squad_responses(
    contexts=example_contexts,
    squad_model=qa_models['squad']['bert'],
    question='Where is Intekglobal located?'
) == ['north of mexico']

assert select_faq_responses(
    question='Is Enrique Jimenez the prettiest person in town?',
    faq_model=qa_models['faq']['tfidf']
) == ['Yes']


assert get_responses(example_contexts, 'What is Intekglobal?',
                     qa_models).strip() == '''
What is Intekglobal?:

0: north of mexico
1: A person.
2: its headquarters
'''.strip()

In [37]:
#export
def get_input(text):
    '''This redundancy is needed for testing'''
    return input(text)


def new_answer(question, data, qa_models):

    if get_input('Give a better anwser [y/n]?')[0].lower() != 'y':
        return 'no data updates..'

    if get_input('Give the answer as a context [y/n]?')[0].lower() == 'y':
        new_context = pd.DataFrame(
            {
                'topic': [get_input('Give context a title:\n')],
                'context': [get_input('Introduce the context:\n')]
            }
        )
        data['context']['df'] = data['context']['df'].append(new_context)
        data['context']['df'].to_csv(data['context']['path'])

        return 'contexts dataset updated..'
    else:
        new_faq = pd.DataFrame(
            {
                'Question': [question],
                'Answer': [get_input('Introduce the answer:\n')]
            }
        )
        data['faq']['df'] = data['faq']['df'].append(new_faq)
        data['faq']['df'].to_csv(data['faq']['path'])
        qa_models['faq']['tfidf'] = train_model(
            configs.faq.tfidf_logreg_en_faq, download=False
        )
        return 'FAQ dataset and model updated..'


def dialog_system(data, qa_models, num_returned_values_per_squad_model=1):
    question = get_input('Introduce question:\n')

    responses = get_responses(
        data['context']['df'], question, qa_models,
        num_returned_values_per_squad_model
    )
    return responses, new_answer(question, data, qa_models)

In [38]:
##Test FAQ dialog system

In [39]:
import tempfile
from unittest.mock import patch
from shutil import copyfile
example_contexts = pd.DataFrame(
    {
        'context':
            [
                'Intekglobal has its headquarters located in TJ',
                'In Intekglobal we care about you',
                'Enrique Jimenez is one of the smartest minds on the planet'
            ]
    },
)

data = {
    'context': {
        'df': example_contexts,
        'path': ...
    },
    'faq': {
        'df': ...,
        'path': ...
    }
}


def copy_data_files(data, tmpdirname):
    data['context']['path'] = path.join(tmpdirname, 'tmp_context.csv')
    data['faq']['path'] = path.join(tmpdirname, 'tmp_faq.csv')
    data['faq']['df'] = pd.read_csv('./faq_data.csv')
    data['context']['df'].to_csv(data['context']['path'])
    copyfile('./faq_data.csv', data['faq']['path'])


@patch('__main__.get_input')
def test_context_response_with_no_updates(mock_input):
    mock_input.side_effect = ['Who is Enrique Jimenez?', 'N']

    with tempfile.TemporaryDirectory() as tmpdirname:
        copy_data_files(data, tmpdirname)
        responses, status = dialog_system(data, qa_models)
        assert 'no data updates..' == status
        assert 'one of the smartest minds on the planet'  in responses


@patch('__main__.train_model')
@patch('__main__.get_input')
def test_updating_faq_dataset(mock_input, mock_train_model):

    new_answer = 'Intekglobal is one of the best companies in the world'
    mock_input.side_effect = ['What is Intekglobal?', 'Y', 'N', new_answer]
    qa_model_faq = qa_models['faq']['tfidf']
    try:

        with tempfile.TemporaryDirectory() as tmpdirname:
            copy_data_files(data, tmpdirname)

            assert 'FAQ dataset and model updated..' == dialog_system(
                data, qa_models
            )[1]

            updated_faq = pd.read_csv(data['faq']['path'])

            assert updated_faq[updated_faq['Answer'] == new_answer
                              ].shape[0] == 1
    except Exception as e:
        print(e)
    finally:
        qa_models['faq']['tfidf'] = qa_model_faq


test_context_response_with_no_updates()
test_updating_faq_dataset()

In [40]:

HOME_DIR = !echo $HOME
HOME_DIR = HOME_DIR[0]

In [41]:
import tempfile
from unittest.mock import patch
from shutil import copyfile, copytree
from pprint import pprint

example_contexts = pd.DataFrame(
    {
        'context':
            [
                'Intekglobal has its headquarters located in TJ',
                'In Intekglobal we care about you'
            ]
    },
)

data = {
    'context': {
        'df': example_contexts,
        'path': ...
    },
    'faq': {
        'df': ...,
        'path': ...
    }
}


def copy_data_files(data, tmpdirname):
    data['context']['path'] = path.join(tmpdirname, 'tmp_context.csv')
    data['faq']['path'] = path.join(tmpdirname, 'tmp_faq.csv')
    data['faq']['df'] = pd.read_csv('./faq_data.csv')
    copyfile('./context_data.csv', data['context']['path'])
    copyfile('./faq_data.csv', data['faq']['path'])


def modify_configs(data, tmpdirname):

    copy_data_files(data, tmpdirname)

    tmp_configs_faq = path.join(tmpdirname, 'temp_config_faq.json')
    tmp_model_dir = path.join(tmpdirname, 'temp_models_dir')

    metadata = json.load(open(configs.faq.tfidf_logreg_en_faq)
                        )['metadata']['variables']

    models_dir = metadata['MODELS_PATH'].replace(
        '{ROOT_PATH}', metadata['ROOT_PATH'].replace('~', HOME_DIR)
    )

    copytree(models_dir, tmp_model_dir)
    copyfile(configs.faq.tfidf_logreg_en_faq, tmp_configs_faq)

    configs.faq.tfidf_logreg_en_faq = tmp_configs_faq

    gen_faq_config_file(
        metadata={'variables': {
            'MODELS_PATH': tmp_model_dir
        }},
        dataset_reader={'data_path': data['faq']['path']}
    )
    #pprint(json.load(open(configs.faq.tfidf_logreg_en_faq)))
    assert path.isdir(tmp_model_dir)
    assert path.isfile(configs.faq.tfidf_logreg_en_faq)


@patch('__main__.get_input')
def test_faq_answer_with_updating(mock_input):

    new_answer = 'Intekglobal is one of the best companies in the world'
    question = 'What is Intekglobal?'
    mock_input.side_effect = [question, 'Y', 'N', new_answer, question, 'N']

    original_config_file = configs.faq.tfidf_logreg_en_faq
    qa_model_faq = qa_models['faq']['tfidf']

    with tempfile.TemporaryDirectory() as tmpdirname:

        try:
            modify_configs(data, tmpdirname)
    
            old_responses = dialog_system(data, qa_models)[0]
            new_responses = dialog_system(data, qa_models)[0]
           
            logging.info(f'Old response:\n {old_responses}')
            logging.info(f'New response:\n{new_responses}')
        except Exception as e:
            print(e)

        finally:
            configs.faq.tfidf_logreg_en_faq = original_config_file
            qa_models['faq']['tfidf'] = qa_model_faq

        assert new_answer not in old_responses
        assert new_answer in new_responses


test_faq_answer_with_updating()

2020-06-09 22:36:36.731 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 203: Loading model sklearn.feature_extraction.text:TfidfVectorizer from /tmp/tmp13etuaio/temp_models_dir/faq/mipt/en_mipt_faq_v4/tfidf.pkl
10:36:36 INFO:Loading model sklearn.feature_extraction.text:TfidfVectorizer from /tmp/tmp13etuaio/temp_models_dir/faq/mipt/en_mipt_faq_v4/tfidf.pkl
2020-06-09 22:36:36.733 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 210: Model sklearn.feature_extraction.textTfidfVectorizer loaded  with parameters
10:36:36 INFO:Model sklearn.feature_extraction.textTfidfVectorizer loaded  with parameters
2020-06-09 22:36:36.734 WARNING in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 216: Fitting of loaded model can not be continued. Model can be fitted from scratch.If one needs to continue fitting, please, look at `warm_start` parameter
10:36:36 WARNING:Fitting of loaded model can not be continu

In [45]:
##Test Context Dialog System
import tempfile
from unittest.mock import patch
from shutil import copyfile, copytree
from pprint import pprint

example_contexts = pd.DataFrame(
    {
        'topic': ['Headquarters', 'Mision'],
        'context':
            [
                'Intekglobal has its headquarters located in TJ',
                'In Intekglobal we care about you'
            ]
    }
)

data = {
    'context': {
        'df': example_contexts,
        'path': ...
    },
    'faq': {
        'df': ...,
        'path': ...
    }
}


def copy_data_files(data, tmpdirname):
    data['context']['path'] = path.join(tmpdirname, 'tmp_context.csv')
    data['faq']['path'] = path.join(tmpdirname, 'tmp_faq.csv')
    data['faq']['df'] = pd.read_csv('./faq_data.csv')
    copyfile('./context_data.csv', data['context']['path'])
    copyfile('./faq_data.csv', data['faq']['path'])


@patch('__main__.get_input')
def test_context_new_answer(mock_input):

    question = 'What is a Chatbot?'
    new_topic = 'AI Tool & Chatbot Development'
    new_context = '''

A chatbot is an important tool for simulating intelligent conversations with humans.
Intekglobal chatbots efficiently live message on platforms such as Facebook Messenger, 
Slack, and Telegram. Assisting consumers with a variety of purposes and industries. 

But chatbots are more than just a cool technology advancement. They actually transform the user experience.
People want simple and convenient interactions with interface and products.

'''

    mock_input.side_effect = [
        question, 'YES', 'yes', new_topic, new_context, question, 'N'
    ]

    with tempfile.TemporaryDirectory() as tmpdirname:

        try:

            copy_data_files(data, tmpdirname)

            old_responses = dialog_system(data, qa_models)[0]
            logging.info(f'Old response:\n {old_responses}')
            new_responses = dialog_system(data, qa_models)[0]
            logging.info(f'New response:\n{new_responses}')
        except Exception as e:
            print(repr(e))

        finally:
            logging.info(' Test finished')

        updated_context = pd.read_csv(data['context']['path'])

        assert updated_context[updated_context['context'] == new_context
                              ].shape[0] == 1

        assert updated_context[updated_context['topic'] == new_topic
                              ].shape[0] == 1
        assert 'an important tool for simulating intelligent conversations with humans' not in old_responses
        assert 'an important tool for simulating intelligent conversations with humans' in new_responses


test_context_new_answer()


10:37:48 INFO:Old response:
 What is a Chatbot?:

0: A person.
1: Intekglobal
2: Intekglobal we care about you

10:37:51 INFO:New response:
What is a Chatbot?:

0: an important tool for simulating intelligent conversations with humans
1: A person.

10:37:51 INFO: Test finished


In [80]:
#export
# if _main_..
def main():
    CONTEXT_DATA_FILE = './context_data.csv'
    FAQ_DATA_FILE = './faq_data.csv'
    qa_models = load_qa_models()

    context = {'df': pd.read_csv(CONTEXT_DATA_FILE), 'path': CONTEXT_DATA_FILE}
    faq = {'df': pd.read_csv(FAQ_DATA_FILE), 'path': FAQ_DATA_FILE}

    data = {'context': context, 'faq': faq}

    while True:
        try:
            dialog_system(data=data, qa_models=qa_models)
        except (KeyboardInterrupt, EOFError, SystemExit):
            logging.info('Goodbye!')
            return 'Goodbye!'

In [84]:
# test main()

from unittest.mock import patch


@patch('__main__.load_qa_models')
@patch('__main__.pd.read_csv')
@patch('__main__.dialog_system')
def test_main_keyboard_interrupt(
    mock_dialog_system,
    mock_pd_read_csv,
    mock_load_qa_models,
):
    mock_dialog_system.side_effect = [
        KeyboardInterrupt(), EOFError(),
        SystemExit()
    ]
    assert 'Goodbye!' == main()
    assert 'Goodbye!' == main()
    assert 'Goodbye!' == main()


test_main_keyboard_interrupt()

11:50:06 INFO:Goodbye!
11:50:06 INFO:Goodbye!
11:50:06 INFO:Goodbye!
